In [1]:
! pip install yfinance


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ms=yf.download('MSFT',start='2012-01-01',end='2024-07-25')
ms.head()

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['MSFT']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [ ]:
train=ms[:'2022'].Close.values.reshape(-1,1)
test=ms['2022':].Close.values.reshape(-1,1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
train=scaler.fit_transform(train)
test=scaler.transform(test)

In [ ]:
def convert_data_extra(data,x_step,y_step):
  X,Y = [],[]
  for i in range(len(data)-x_step-y_step+1):
    d=i+x_step

    X.append(data[i:d])
    Y.append(data[d:d+y_step])

  return np.array(X),np.array(Y)

x_step=10
y_step=1
trainX,trainY=convert_data_extra(train,x_step,y_step)
testX,testY=convert_data_extra(test,x_step,y_step)

trainX.shape,testX.shape

In [ ]:
trainX=np.reshape(trainX,(trainX.shape[0],trainX.shape[1],1))
testX=np.reshape(testX,(testX.shape[0],testX.shape[1],1))

trainX.shape,testX.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,LSTM,Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import LSTM,Input
from tensorflow.keras import Model

In [ ]:
from tensorflow.keras.layers import LSTM

In [ ]:
trainX.shape[1:]

In [ ]:
model = Sequential()
model.add(Input(shape=trainX.shape[1:]))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(Bidirectional(LSTM(64,return_sequences=False)))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

In [ ]:
history=model.fit(trainX,trainY,epochs=50,batch_size=16)

In [ ]:
train_preds=model.predict(trainX)
test_preds=model.predict(testX)
predicted=np.concatenate((train_preds,test_preds),axis=0)

In [ ]:
plt.plot(ms.Close.values,label='real')
plt.plot(scaler.inverse_transform(predicted),label='predicted')
plt.axvline(x=len(train),c='r')
plt.legend()
plt.show()